In [7]:
# login required for the first time running this notebook
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from datasets import Dataset
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

In [2]:
from transformers import SpeechT5ForTextToSpeech
from datasets import load_dataset, load_from_disk
import numpy as np
from IPython.display import Audio,display
import os

In [8]:
# load model from huggingface or local if found
audio_model_path = 'audio_model'
if os.path.exists(audio_model_path):
    print("Audio model found")
    model = SpeechT5ForTextToSpeech.from_pretrained('audio_model')
else:
    print("Audio model not found, fetching from hugging face")
    model = SpeechT5ForTextToSpeech.from_pretrained("Nour17/speecht5_finetuned_Andrew_NG_complete")
    model.save_pretrained('audio_model')

Audio model not found, fetching from hugging face


NameError: name 'SpeechT5ForTextToSpeech' is not defined

In [4]:
# load dataset from huggingface or local if found
dataset_model_path = 'audio_dataset'
if os.path.exists(dataset_model_path):
    print("Audio dataset found")
    dataset = load_from_disk('audio_dataset')
else:
    print("Audio dataset not found, fetching from hugging face")
    dataset = load_dataset("Nour17/Andrew_NG_audio_dataset")
    dataset.save_to_disk('audio_dataset')

Audio dataset found


In [5]:
# default speaker embeddings
import torch
example = dataset["test"][0]
default_speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [6]:
# normalize input text 
from nemo_text_processing.text_normalization.normalize import Normalizer
normalizer = Normalizer(input_case='cased', lang='en')

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.


In [7]:
# convert mel-spectogram to audio waveform model (reverse fft phase shift)
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [8]:
# get sentences and add period at the end of each one
def get_sentences(text, add_period = True):
    period = "." if add_period else ""
    sentences = [sent + period for sent in text.split(".")]
    # remove last period if it exsists and add_period flag is True
    if sentences[-1] == period : sentences = sentences[:-1]
    return sentences

# divide large text into sentences chunks
def chunk_text(text, max_length = 400):
    sentences = get_sentences(text)
    chunks = [[]]
    chunks_idx = 0
    curr_length = 0
    for sent in sentences:
        curr_length += len(sent)
        if (curr_length < max_length):
            chunks[chunks_idx].append(sent)
        else:
            chunks.append([sent])
            chunks_idx += 1
            curr_length = len(sent)
    return chunks

In [9]:
def prepare_sentences(sentences):
    sentences = normalizer.normalize_list(sentences)
    inputs = processor(text=sentences,is_split_into_words=True, return_tensors="pt")
    return inputs

def generate_single_speech(text):
    if type(text) == list :
        inputs = prepare_sentences(text)
        speech_tensor = model.generate_speech(inputs["input_ids"], default_speaker_embeddings, vocoder=vocoder)
    elif type(text) == str :
        sentences = get_sentences(text)
        print(sentences)
        inputs = processor(text=normalizer.normalize(sentences[0]), return_tensors="pt") if len(sentences) == 1 else prepare_sentences(sentences)
    speech_tensor = model.generate_speech(inputs["input_ids"], default_speaker_embeddings, vocoder=vocoder)
    return speech_tensor.numpy()

def generate_speech(text):
    transcripts = chunk_text(text)
    speech_tensor_list = []
    for transcript in transcripts:
        print(transcript)
        speech_tensor_list.append(generate_single_speech(transcript))
    return np.concatenate(speech_tensor_list)

In [39]:
#from pydub import AudioSegment
from moviepy.editor import ImageClip,AudioFileClip, concatenate_videoclips
from moviepy.audio.AudioClip import AudioArrayClip
import matplotlib.pyplot as plt

class Slide:
    def __init__(self, title, bullet_points, audio_array,audio_path):
        self.title = title
        self.bullet_points = bullet_points
        # reshape audio array to be size (N x 1) if necessary
        self.audio_array = audio_array if len(audio_array.shape) == 2 else audio_array.reshape(audio_array.shape[0],1)
        self.audio_path = audio_path

    def generate_img(self):
        fig, ax = plt.subplots()
        ax.set_title(self.title)
        for idx, bp in enumerate(self.bullet_points):
            ax.text(0.05, 0.9 - idx * 0.1, f"- {bp}", fontsize=12, transform=ax.transAxes)
        ax.axis('off') 

        # Convert Matplotlib figure to a NumPy array
        fig.canvas.draw()
        img_data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img_data = img_data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.close()
        return img_data

    """def get_slide_duration(self):
        audio = AudioSegment.from_file(self.audio_path)
        return len(audio)"""
    
    @staticmethod
    def generate_video_from_slides(slides,video_name):
        clips = []
        for slide in slides:
            #slide_duration = slide.get_slide_duration()
            slide_img = slide.generate_img()
            #audio_clip = AudioArrayClip(slide.audio_array,fps = 32000)
            audio_clip = AudioFileClip(slide.audio_path)
            slide_clip = ImageClip(slide_img,duration = audio_clip.duration)
            slide_clip = slide_clip.set_audio(audio_clip)
            print(slide_clip.duration)
            clips.append(slide_clip)
        final_clip = concatenate_videoclips(clips)
        # export video
        final_clip.write_videofile(video_name, fps=24)
        

In [30]:
# smapling rate constant
sr = 16000

In [31]:
from scipy.io.wavfile import write
def prediction(title: str,points:[str],text:str):
    speech = generate_speech(text)
    filename = title + '.wav'
    write(filename, sr, speech)
    slide = Slide(title,points,speech,filename)
    Slide.generate_video_from_slides([slide])

In [ ]:
title = "test title"
points = ["100","200","300"]
text = "first is 100. second is 200. 3rd is 300. no more testing required."
prediction(title,points,text)

In [6]:
import os
currDir = os.getcwd()
currDirList = currDir.split('/')
if(currDirList[-1] == "notebooks"):
    newDir = "/".join(currDirList[:-1])
    print(newDir)

/home/crispy_shawarma/Graduation Project/AI-Service


In [32]:
def connect_azure_blob(acc_name,acc_key):
    # change curr directory to root if necessary
    currDir = os.getcwd()
    currDirList = currDir.split('/')
    if(currDirList[-1] == "notebooks"):
        newDir = "/".join(currDirList[:-1])
        os.chdir(newDir)
    from src.utils.azure_utils import AzureStorage
    azure_storage_object = AzureStorage(acc_name,acc_key)
    # return to current directory
    os.chdir(currDir)
    return azure_storage_object

In [60]:
# create pydantic slides
from pydantic import BaseModel, RootModel

class SlidePydantic(BaseModel):
    title: str
    points: list[str]
    text: str
class ListSlidesPydantic (BaseModel):
    slides: list[SlidePydantic]
    def __iter__(self):
        return iter(self.slides)

    def __getitem__(self, item):
        return self.slides[item]

In [34]:
# storing blob name:
def store_blob(video_name,video_path):
    blob_name = datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + video_name
    azure_storage_object.store(container_name,blob_name,video_path)
    return blob_name

In [35]:
# creating sas link

from datetime import datetime, timedelta
from azure.storage.blob import BlobClient, generate_blob_sas, BlobSasPermissions
def get_blob_sas(container_name, blob_name):
    sas_blob = generate_blob_sas(account_name=acc_name, 
                                container_name=container_name,
                                blob_name=blob_name,
                                account_key=acc_key,
                                permission=BlobSasPermissions(read=True),
                                expiry=datetime.utcnow() + timedelta(hours=1))
    return sas_blob

In [61]:
import uvicorn
from fastapi import FastAPI


app = FastAPI(title='TTS_model')

#setup azure blob connection

acc_name = "graduationproject19024"
acc_key = "l6cWAFptkSjT9EI033DehgiVbVXojBFF89mAN+JN8ibNGk3jhB/TComT+FEf+w3YvuQoWSu7TGbT+AStQWWNZg=="
container_name = "videos"
azure_storage_object = connect_azure_blob(acc_name,acc_key)

# Get method endpoint
@app.get("/")
def home():
    return "API is working correctly. Now head over to http://localhost:8000/docs."


# post tts model endpoint
@app.post("/tts") 
def generate_video(slides:ListSlidesPydantic):
    generated_slides = []
    i = 1
    for slide in slides:
        print(type(slide))
        #generate speech
        speech = generate_speech(slide.text)
        filename = f'slide {i}.wav'
        i = i +1
        write(filename, sr, speech)
        # generate slides
        slide = Slide(slide.title,slide.points,speech,filename)
        generated_slides.append(slide)
    video_name = "output_video.mp4"
    video_path = video_name
    Slide.generate_video_from_slides(generated_slides,video_name)
    # create blob and store it on azure
    blob_name = datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + video_name
    azure_storage_object.store(container_name,blob_name,video_path)
    # get sas for blob for public access
    blob = get_blob_sas(container_name, blob_name)
    url = 'https://'+acc_name+'.blob.core.windows.net/'+container_name+'/'+blob_name+'?'+blob
    # Return file url
    return url

In [62]:
import nest_asyncio

# Allows the server to be run in this interactive environment
nest_asyncio.apply()

host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [7247]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


<class '__main__.SlidePydantic'>
['first is 100.', ' second is 200.', ' 3rd is 300.', ' more testing required.']


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 72.31it/s]


<class '__main__.SlidePydantic'>
['fourth is 400.', ' fifth is 500.', ' 6th is 600.', ' no more testing required.']


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 67.68it/s]


6.43
6.21
Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
INFO:     127.0.0.1:54546 - "POST /tts HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7247]


KeyboardInterrupt: 